In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

**read_data_small** is the function to read in the small dataset about 30 MB

In [2]:
def read_data_small():
    X_train = pd.read_csv("data_small/X_train_small.csv")
    X_test = pd.read_csv("data_small/X_test_small.csv")
    y_train = np.asarray(pd.read_csv("data_small/y_train_small.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data_big** is the function to read in the big dataset about 100 MB

In [3]:
def read_data_big():
    X_train = pd.read_csv("data_big/X_train_big.csv")
    X_test = pd.read_csv("data_big/X_test_big.csv")
    y_train = np.asarray(pd.read_csv("data_big/y_train_big.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data** is the function to read in the whole dataset about 1.5 G

In [4]:
def read_data():
    X_train = pd.read_csv("data/X_train.csv")
    X_test = pd.read_csv("data/X_test.csv")
    y_train = np.asarray(pd.read_csv("data/y_train.csv", header=None)[0])
    return X_train, X_test, y_train

# Insert Your Code Here

In [5]:
%%time
X_train, X_test, y_train = read_data_small()
data = X_train

CPU times: user 752 ms, sys: 112 ms, total: 864 ms
Wall time: 870 ms


In [6]:
#type1and2 = data[(data["class"] == 1) ^ (data["class"] == 2)]
type1and2 = data

In [19]:
sort_by = ["member", "user", "endUserRef","obId"]
type1and2_person = np.unique(type1and2[sort_by].values.astype(str), axis=0)

In [20]:
all_person = data[sort_by].values.astype(str)
type1and2_person_activities = data[[x in type1and2_person for x in all_person]]

In [21]:
len(data)

141822

In [22]:
len(type1and2_person)

479

In [36]:
y_val = pd.DataFrame()
y_val["index"] = data.index
y_val["true_class"] = y_train
y_val.set_index("index")

y_pred = pd.DataFrame()
y_pred["index"] = data.index
y_pred["predicted_class"] = 0
y_pred.set_index("index")
print("")

In [ ]:
%%time
a = type1and2_person_activities.groupby(["member", "user", "endUserRef","obId"])
short_time = 1000

for k in range(len(type1and2_person)):
    if k % 10 == 0:
        print("k = ",k)
    
    person_0 = a.get_group(tuple(type1and2_person[k]))[["timestamp","type","volume", "operation", "isBid", "orderId",'isBuyer','bidOrderId', 'askOrderId', "isAggressor"]]
    if len(person_0) > len(data) /500:
        continue
    
    person_0_orderbook = {}
    volume_ls = []
#    class_ls = []
    for x in person_0.iterrows():
        action = x[1]
        if action.type == "ORDER":
            pos = int(action.volume)
            if action.isBid == False:
                pos = pos * -1
            if action.operation == "CANCEL":
                del person_0_orderbook[action.orderId]
#                print("CANCEL", pos, end=",  ")
            else:
                person_0_orderbook[action.orderId] = pos
#            if action.operation == "INSERT":
#                print("INSERT", pos, end=",  ")            
            volume = list(person_0_orderbook.values())
            volume_ls.append(volume)
#            class_ls.append(action['class'])
#            print(x[0], volume, "true class", action['class'], "predict class", -(x[0] in v))
        else:
            pos = int(action.volume)
            if action.isBuyer == False:
                pos = pos * -1
#            print("TRADE", pos, x[0], "true class", action['class'], "predict class", -(x[0] in v))

    volume_history = pd.DataFrame()
    volume_history['time'] = person_0[person_0["type"]=="ORDER"].index
    volume_history['volume'] = volume_ls
    volume_history['long_pos'] = [[y for y in x if y>0] for x in volume_ls]
    volume_history['short_pos'] = [[y for y in x if y<0] for x in volume_ls]
    volume_history['long_pos_total'] = [sum([y for y in x if y>0]) for x in volume_ls]
    volume_history['short_pos_toal'] = [sum([y for y in x if y<0]) for x in volume_ls]
    volume_history = volume_history.set_index('time')
    volume_threshold = np.mean([abs(y) for x in volume_ls for y in x]) * 0.8
    #volume_history
    #plt.hist([abs(y) for x in volume_ls for y in x])

    canceled_order = person_0[(person_0["operation"] == "CANCEL") & (person_0["volume"] != 0)]
    person_0_trades = person_0[person_0["type"]=="TRADE"]
    person_0_insert = person_0[(person_0["operation"]=="INSERT") & (person_0["orderId"].isin(canceled_order['orderId'].values))]

    canceled_order_time = pd.DataFrame()
    canceled_order_time["endtime"] = canceled_order.index
    canceled_order_time["orderId"] = canceled_order.orderId.values.astype(str)
    canceled_order_time["isBid"] = canceled_order.isBid.values
    person_0_insert_time = pd.DataFrame()
    person_0_insert_time["starttime"] = person_0_insert.index
    person_0_insert_time["orderId"] = person_0_insert.orderId.values.astype(str)

    canceledordertimes = pd.concat([person_0_insert_time.set_index("orderId"),canceled_order_time.set_index("orderId")], axis=1, join='inner')
    canceled_order = pd.concat([canceledordertimes,canceled_order.set_index("orderId")], axis=1, join='inner')

    start = np.array(canceled_order['starttime'].values)
    end = np.array(canceled_order['endtime'].values)
    trade_ind = np.array(person_0_trades.index)

    ls = set()
    #Can be optimized
    for i in range(len(start)):
        for j in range(len(trade_ind)):
    #        print(i, j, trade_ind[j] > start[i] ,trade_ind[j] < end[i] , end[i] - trade_ind[j] < 1000 )
            if trade_ind[j] > start[i] and trade_ind[j] < end[i] and end[i] - trade_ind[j] < short_time and end[i]-start[i] < short_time * 5:
    #            print("One")
    #            print([trade_ind[j],start[i], end[i]])
    #            print(abs(canceled_order.iloc[i].volume), volume_threshold)
                if abs(canceled_order.iloc[i].volume) > 0.55*volume_threshold and person_0_trades.iloc[j]['isBuyer'] != canceledordertimes.iloc[i]["isBid"] and person_0_trades.iloc[j]['isAggressor'] == False:
    #                print("Two")
                    id = (person_0_trades.iloc[j].bidOrderId if person_0_trades.iloc[j]['isBuyer']==True else person_0_trades.iloc[j].askOrderId)
                    sold = person_0[(person_0["operation"]=="INSERT") & (person_0["orderId"] == id)]
    #                print(sold.index)
                    other_trade_index = person_0_trades.iloc[np.where([(person_0_trades["bidOrderId"] == id) ^ (person_0_trades["askOrderId"] == id)])[1]].index
                    ls.update([trade_ind[j],start[i], end[i]], sold.index,other_trade_index)
    #                print("ls", ls)

    v = np.array(list(ls))
    v.sort()


#    w = np.array(person_0[person_0["class"]!=0].index)

#    print([x for x in v if x not in w], [x for x in w if x not in v], len(w), len(v))
    

   
    action_ls = pd.DataFrame()
    action_ls['index'] = v
    action_ls['action'] = ""
    action_ls['predicted_class'] = 0
    action_ls = action_ls.set_index("index")
    action_ls
    for x in person_0.loc[v].iterrows():
        action = x[1]
        if action.type == "ORDER":
            if action.operation == "CANCEL":
                action_ls['action'].loc[x[0]] = "CANCEL"
            if action.operation == "INSERT":
                action_ls['action'].loc[x[0]] = "INSERT"
        else:
            action_ls['action'].loc[action.name] = "TRADE"

    c_count = 0
    i_count = 0
    c_ls = []
    for x in action_ls[::-1].iterrows():        
        if x[1].action!="CANCEL":
            if c_count!=0:
                c_ls.append(x[0])
                if x[1].action == "INSERT":
                    i_count += 1
                    if i_count == (c_count + 1):
                        if c_count == 1:
                            for y in c_ls:
                                action_ls['predicted_class'].loc[y]=1
                        else:
                            for y in c_ls:
                                action_ls['predicted_class'].loc[y]=2
                        i_count = 0
                        c_count = 0
                        c_ls.clear()
        else:
            c_ls.append(x[0])
            c_count += 1
    action_ls = action_ls.replace(to_replace=0, method='ffill')

    if len(v) < 1000:
        for x in action_ls.index:
            y_pred.loc[x] = action_ls.loc[x]["predicted_class"]

k =  0
k =  10
k =  20
k =  30
k =  40
k =  50
k =  60
k =  70
k =  80
k =  90
k =  100
k =  110
k =  120
k =  130
k =  140
k =  150
k =  160
k =  170
k =  180
k =  190
k =  200
k =  210


In [55]:
from sklearn.metrics import cohen_kappa_score

def score(y_pred, y_true):
    y_pred_label = np.argmax(y_pred, axis=1)
    return cohen_kappa_score(y_pred_label, y_true)

In [54]:
y_true = y_val["true_class"]

ls = []
t = []
for x in y_pred["predicted_class"].values:
    t = np.zeros(4,int)
    t[x] = 1
    ls.append(t)
y_pred_label = np.array(ls)
score(res, y_true)

0.9521952899182705

0.9521952899182705

In [42]:
np.count_nonzero(np.subtract(y_true,y_pred_label)), np.count_nonzero(y_true), len(y_true)

(51, 533, 141822)

### Submission Format

The classifier function wrote should return a 4d nparray with 4 columns. The columns are corresponding to the class labels: 0, 1, 2, 3. Please see examples below.

In [ ]:
y_train_prob_pred

In [ ]:
y_test_prob_pred

### Write test results to csv files

Please rename your file to indicate which data set you are working with. 

- If you are using the small dataset: *y_train_prob_pred_small.csv* and *y_test_prob_pred_small.csv*
- If you are using the small dataset: *y_train_prob_pred_big.csv* and *y_test_prob_pred_big.csv*
- If you are using the original dataset: *y_train_prob_pred.csv* and *y_test_prob_pred.csv*

In [ ]:
pd.DataFrame(y_train_prob_pred).to_csv("y_train_prob_pred.csv")
pd.DataFrame(y_test_prob_pred).to_csv("y_test_prob_pred.csv")